In [2]:
#Code to mount Google Drive at Colab Notebook instance
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Workaround to avoid following error at notebook
# NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# **Install all required libraries**

1. Huggingface libraries to use Mistral 7B LLM (Open Source LLM)

2. LangChain library to call LLM to generate reposne based on the prompt

In [4]:
# Huggingface libraries to run LLM.
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

#LangChain related libraries
!pip install -q -U langchain

#Open-source pure-python PDF library capable of splitting, merging, cropping,
#and transforming the pages of PDF files
!pip install -q -U pypdf

#Python framework for state-of-the-art sentence, text and image embeddings.
!pip install -q -U sentence-transformers

# FAISS Vector Databses specific Libraries
!pip install -q -U faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.6/218.6 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.4 MB/s et

Import required Libraries and check whether we have access to GPU or not.

We must be getting following output after running the cell

Device: cuda

Tesla T4

In [7]:
#from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig

import torch

from langchain.llms import HuggingFacePipeline

from langchain.chains import ConversationalRetrievalChain

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.vectorstores import FAISS

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))


Device: cuda
Tesla T4



# **We are using Mistral 7 Billion parameter LLM (Open source from HuggingFace)**

If you're new to Hugging Face or machine learning tasks, the following lines of code introduce some scary concepts. This Mistral 7B LLM, although big in size (approximately 14 GB), presents a challenge when loading on a Colab notebook, requiring larger GPUs and making it incompatible with the free T4 GPU.

To address this issue, we employ a sharded version of the Mistral LLM. This version is loaded in smaller increments (approximately 1.9 GB each) in the notebook and subsequently aggregated into a single file. This allows us to seamlessly utilize the LLM on a free Colab GPU (T4 GPU) without incurring any costs for learning purposes.

These lines of code incorporate several important concepts, which may initially seem complex but are easily comprehensible:

1. BitsAndBytesConfig:

  - load_in_4bit
  - bnb_4bit_use_double_quant
  - bnb_4bit_quant_type
  - bnb_4bit_compute_dtype

2. AutoModelForCausalLM.from_pretrained

3. AutoTokenizer.from_pretrained

Despite the use of some technical terminology, don't be discouraged. In the coming days, we'll delve into these concepts in an approachable manner, breaking down the complexities. This includes a detailed understanding of the mentioned configurations, loading procedures, and tokenization processes. Embrace the learning journey; it's simpler than it seems.








In [6]:
origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

pytorch_model-00002-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00003-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

# **Creating pipelines to run LLM at Colab notebook**

It uses

1. Huggingface transformers pipeline
  - This is an object that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering.

  - We are using "text-generation" task. It's sort of chatGPT where you ask question and model respond with answer of that question based on it's intelligence.


2. LangChain HuggingFacePipeline
  - Integrating "transformers.pipeline" with LangChain.
  - You may be thinking why we need this. We will be using LangChain extensively later and this is first step to integrate our LLM with LangChain


In [8]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=300,
    temperature = 0.3,
    do_sample=True,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# **Load the PDF and create Chunk of texts**


In [9]:
# Load the pdf file
loader = PyPDFLoader('/content/drive/MyDrive/Colab Notebooks/ImpactofIndianPremierLeagueonTestCricketinIndia.pdf')
documents = loader.load()

# Split the documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunked_docs  = text_splitter.split_documents(documents)

In [11]:
# HuggingFace embeddings to create embedding of chunked docs to store at
#Vector Database

embeddings = HuggingFaceEmbeddings()

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [12]:
# Store chunk of pdf files at FAISS vector database by using HuggingFace Embedding model


faiss_db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [13]:
# Connect query to FAISS index using a retriever
retriever = faiss_db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [14]:
# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(mistral_llm, retriever,return_source_documents=True)

In [16]:
#Get the answer of question from Vector Database
import sys

def get_user_input():
    return input('Prompt: ').lower()

def main():
    chat_history = []

    query = get_user_input()

    result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
    print(f'Answer: {result["answer"]}\n')

    chat_history.append((query, result['answer']))

if __name__ == "__main__":
    main()

Prompt: Who won first match in IPL?
Answer:  The first match in IPL was played between Kolkata Knight Riders and Bangalore Royal Challengers on April 9, 2008.
Answer: The first match in IPL was played between Kolkata Knight Riders and Bangalore Royal Challengers on April 9, 2008.

